[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/_HOWTOs/paraview_hello_world.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/_HOWTOs/paraview_hello_world.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/_HOWTOs/paraview_hello_world.ipynb)

### TODO #1417 add description
### TODO #1490 fix installation on binder
For more informations about Paraview please refere to [Paraview documentation](https://docs.paraview.org/en/latest/Tutorials/SelfDirectedTutorial/)

In [7]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [8]:
import os
import subprocess
SUBPROCESS_ENV = os.environ.copy()

if 'google.colab' in sys.modules:
    !apt-get install -qq ghostscript
    !wget -nv "https://paraview.org/paraview-downloads/download.php?submit=Download&version=v5.13&type=binary&os=Linux&downloadFile=ParaView-5.13.1-egl-MPI-Linux-Python3.10-x86_64.tar.gz" -O paraview.tar.gz
    !tar xzf paraview.tar.gz
    SUBPROCESS_ENV['PATH'] += ':' + subprocess.check_output(['bash', '-c', "echo `pwd`/`dirname ParaView*/bin/pvpython`"], text=True)[:-1]
    
    # check if Ghostscript's ps2pdf works
    assert subprocess.check_call(['type', 'ps2pdf'], shell=True) == 0
    
# check if Paraview's pvpython works
assert subprocess.check_call(['pvpython', '--version'], env=SUBPROCESS_ENV) == 0
assert subprocess.check_call(['pvpython', '-c', 'import paraview'], env=SUBPROCESS_ENV) == 0

In [11]:
from PySDM_examples.Arabas_et_al_2015 import Settings, SpinUp
from PySDM_examples.utils.kinematic_2d import Simulation, Storage
from PySDM.exporters import VTKExporter
from PySDM_examples.utils import ProgBarController
from PySDM.physics import si
from PySDM import products as PySDM_products
import PySDM_examples
import glob
import platform
import pathlib

In [2]:
products = [
    PySDM_products.EffectiveRadius(unit='um'),
    PySDM_products.FlowVelocityComponent(component = 0, name = 'cx'),
    PySDM_products.FlowVelocityComponent(component = 1, name = 'cy')
]

##### 1. run a simulations saving output to VTK files

In [ ]:
settings = Settings()
settings.simulation_time = 100 * si.minute
storage = Storage()
simulation = Simulation(settings, storage, SpinUp=SpinUp)
simulation.reinit(products)

vtk_exporter = VTKExporter(path='.')    

simulation.run(ProgBarController("progress:"), vtk_exporter=vtk_exporter)
vtk_exporter.write_pvd()

FloatProgress(value=0.0, description='progress:', max=1.0)

#### 2. execute ``PySDM_examples/utils/pvanim.py`` script using `pvpython`

In [9]:
pvanim = pathlib.Path(PySDM_examples.__file__).parent / "utils" / "pvanim.py"
result = subprocess.run(["pvpython", str(pvanim), '--help'], check=True, env=SUBPROCESS_ENV)

In [10]:
product = pathlib.Path("./output/sd_products.pvd").absolute()
attributes = pathlib.Path("./output/sd_attributes.pvd").absolute()

try:
    for mode in ('light', 'dark'):
        result = subprocess.run(
            [
                "pvpython",
                "--force-offscreen-rendering",
                str(pvanim),
                str(product),
                str(attributes),
                str(pathlib.Path('./output').absolute()),
                "--animationname", "docs_intro_animation.ogv",
                "--mode", mode,
            ] + (["--animationframename", "last_animation_frame.pdf"] if mode == 'light' else []),
            check=platform.system() != "Windows",
            capture_output=True,
            text=True,
            env=SUBPROCESS_ENV,
        )
except subprocess.CalledProcessError as e:
    print(e.stderr)
    assert False

#### 3. reduce file size for generated pdf files

In [16]:
if platform.system() != 'Windows':
    for file in glob.glob('output/anim_frame_*.pdf'):
        subprocess.run(['ps2pdf', file, file+'_'], capture_output=True, check=True)
        subprocess.run(['mv', file+'_', file], check=True)